In [1]:
import os
import json
from IPython.display import JSON
from unstructured.partition.auto import partition  
from unstructured.partition.html import partition_html 
from unstructured.chunking.title import chunk_by_title 
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain.prompts.prompt import PromptTemplate
from langchain_openai import OpenAI
from langchain.chains import ConversationalRetrievalChain, LLMChain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from dotenv import load_dotenv
import os


# Load environment variables from the .env file
load_dotenv(dotenv_path=r"C:/Users/gabri/Desktop/Rag_Project/.env.txt")

# Now you can access your API keys like this
openai_api_key = os.getenv("OPENAI_API_KEY")


In [2]:
urls = ["https://msutexas.edu/about/", 
                    "https://msutexas.edu/housing/meal-plan.php", 
                   "https://msutexas.edu/academics/index.php",
                   "https://catalog.msutexas.edu/content.php?catoid=41&navoid=2289",
                    "https://catalog.msutexas.edu/content.php?catoid=41&navoid=2311",
                    "https://catalog.msutexas.edu/preview_program.php?catoid=41&poid=5679",
                    "https://catalog.msutexas.edu/content.php?catoid=41&navoid=2254",
                    "https://catalog.msutexas.edu/content.php?catoid=41&navoid=2312",
                    "https://catalog.msutexas.edu/content.php?catoid=41&navoid=2282",
                    "https://catalog.msutexas.edu/content.php?catoid=41&navoid=2287",
                    "https://catalog.msutexas.edu/content.php?catoid=41&navoid=2316",
                    "https://catalog.msutexas.edu/content.php?catoid=41&navoid=2256",
                    "https://catalog.msutexas.edu/preview_program.php?catoid=41&poid=5679",
                    "https://catalog.msutexas.edu/preview_program.php?catoid=41&poid=5679",
                    "https://catalog.msutexas.edu/preview_program.php?catoid=41&poid=5679"]

In [3]:
elements_url = partition_html(url = "https://msutexas.edu/about/")

elements_dict = [element.to_dict() for element in elements_url]

json_string = json.dumps(elements_dict, indent=2)

print(json_string)



[
  {
    "type": "Title",
    "element_id": "2431404cb722fbecca94eccb81221cb0",
    "text": "About MSU Texas",
    "metadata": {
      "category_depth": 0,
      "languages": [
        "eng"
      ],
      "filetype": "text/html",
      "url": "https://msutexas.edu/about/"
    }
  },
  {
    "type": "Title",
    "element_id": "0cf2cdf3925171217b865d7c471fdc2a",
    "text": "Why MSU Texas",
    "metadata": {
      "category_depth": 1,
      "languages": [
        "eng"
      ],
      "filetype": "text/html",
      "url": "https://msutexas.edu/about/",
      "parent_id": "2431404cb722fbecca94eccb81221cb0"
    }
  },
  {
    "type": "NarrativeText",
    "element_id": "4c3d658bf905baf7a19484563f30e563",
    "text": "Midwestern State University (MSU Texas) is a public university in Wichita Falls, Texas. We are a small and mighty community of Mustangs, with an average class size of just 30 students, 75+ degree programs to choose from, and an opportunity-rich location halfway between Oklahom

In [4]:
for url in urls:
    elements_url += partition_html(url = url)
   

In [18]:
elements_url[0].metadata.__dict__

{'category_depth': 0,
 'languages': ['eng'],
 '_known_field_names': frozenset({'attached_to_filename',
            'bcc_recipient',
            'category_depth',
            'cc_recipient',
            'coordinates',
            'data_source',
            'detection_class_prob',
            'detection_origin',
            'email_message_id',
            'emphasized_text_contents',
            'emphasized_text_tags',
            'file_directory',
            'filename',
            'filetype',
            'header_footer_type',
            'image_base64',
            'image_mime_type',
            'image_path',
            'is_continuation',
            'key_value_pairs',
            'languages',
            'last_modified',
            'link_start_indexes',
            'link_texts',
            'link_urls',
            'links',
            'orig_elements',
            'page_name',
            'page_number',
            'parent_id',
            'sent_from',
            'sent_to',
       

In [22]:
chunks = chunk_by_title(
    elements_url,
    combine_text_under_n_chars=100,
    max_characters=3000,
)

In [24]:
documents = []
for element in chunks:
    metadata = element.metadata.to_dict()
    del metadata["languages"]
    metadata["source"] = metadata["url"]
    documents.append(Document(page_content=element.text, metadata=metadata))

In [29]:
# List of keys to remove
keys_to_remove = ["filetype", "link_texts", "link_urls", "url", "orig_elements", "is_continuation", "text_as_html", 'emphasized_text_contents','emphasized_text_tags']

# Iterate over all documents and clean metadata
for doc in documents:
    if hasattr(doc, "metadata") and isinstance(doc.metadata, dict):  # Ensure metadata is a dictionary
        for key in keys_to_remove:
            doc.metadata.pop(key, None)  # Remove key if it exists

# Print cleaned metadata for verification
for doc in documents:
    print(doc.metadata)  # Check if metadata is cleaned

{'source': 'https://msutexas.edu/about/'}
{'source': 'https://msutexas.edu/about/'}
{'source': 'https://msutexas.edu/about/'}
{'source': 'https://msutexas.edu/about/'}
{'source': 'https://msutexas.edu/about/'}
{'source': 'https://msutexas.edu/about/'}
{'source': 'https://msutexas.edu/about/'}
{'source': 'https://msutexas.edu/about/'}
{'source': 'https://msutexas.edu/about/'}
{'source': 'https://msutexas.edu/about/'}
{'source': 'https://msutexas.edu/about/'}
{'source': 'https://msutexas.edu/about/'}
{'source': 'https://msutexas.edu/about/'}
{'source': 'https://msutexas.edu/about/'}
{'source': 'https://msutexas.edu/about/'}
{'source': 'https://msutexas.edu/about/'}
{'source': 'https://msutexas.edu/about/'}
{'source': 'https://msutexas.edu/about/'}
{'source': 'https://msutexas.edu/about/'}
{'source': 'https://msutexas.edu/about/'}
{'source': 'https://msutexas.edu/about/'}
{'source': 'https://msutexas.edu/about/'}
{'source': 'https://msutexas.edu/about/'}
{'source': 'https://msutexas.edu/a

In [32]:
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(documents, embeddings)

In [33]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 6}
)

In [34]:
template = """You are an AI assistant for answering questions about Midwestern State University(MSU).
You are given all the information about MSU available on the net and a question. Provide a conversational answer.
If you don't know the answer, just say "Hmm, I'm not sure." Don't try to make up an answer.
Question: {question}
=========
{context}
=========
Answer in Markdown:"""
prompt = PromptTemplate(template=template, input_variables=["question", "context"])

In [35]:
llm = OpenAI(temperature=0)

doc_chain = load_qa_with_sources_chain(llm, chain_type="map_reduce")
question_generator_chain = LLMChain(llm=llm, prompt=prompt)
qa_chain = ConversationalRetrievalChain(
    retriever=retriever,
    question_generator=question_generator_chain,
    combine_docs_chain=doc_chain,
)

C:\Users\gabri\AppData\Local\Temp\ipykernel_17184\3323193037.py:3: LangChainDeprecationWarning: This function is deprecated. Refer to this guide on retrieval and question answering with sources: https://python.langchain.com/docs/how_to/qa_sources/
See also the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

  doc_chain = load_qa_with_sources_chain(llm, chain_type="map_reduce")
C:\Users\gabri\AppData\Local\Temp\ipykernel_17184\3323193037.py:4: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  question_

In [37]:
qa_chain.invoke({
    "question": "is there counseling at MSU",
    "chat_history": []
})["answer"]

' Yes, there is counseling available at MSU through the Student Support Services program and the Office of Tutoring and Academic Support Programs. \nSOURCES: https://catalog.msutexas.edu/content.php?catoid=41&navoid=2282'